In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [ ]:
## load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

In [ ]:
## preprocess the data
## Drop unnecessary columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

In [ ]:
## encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

In [ ]:
## one-hot encode the 'Geography' column
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geo = OneHotEncoder()
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

In [ ]:
geo_encoder.toarray()

In [ ]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

In [ ]:
geo_encoder_df=pd.DataFrame(geo_encoder.toarray(), columns=one_hot_encoder_geo.get_feature_names_out(['Geography'])) 
geo_encoder_df

In [ ]:
## combine the one-hot encoded columns with the original dataframe
data = pd.concat([data.drop(['Geography'], axis=1), geo_encoder_df], axis=1)
data.head()

In [ ]:
## Save the encoders and sscaler for later use
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)
with open('one_hot_encoder_geo.pkl', 'wb') as f:
    pickle.dump(one_hot_encoder_geo, f)


In [ ]:
data.head()

In [ ]:
## Divide the data into independent and dependent features
X = data.drop('Exited', axis=1)
y = data['Exited']

## Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
## Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

In [ ]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

ANN Implementation

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [ ]:
X_train.shape[1]

In [ ]:
## Build the ANN model
model = Sequential([
    Dense(64,activation='relu', input_shape=(X_train.shape[1],)), ##HL1
    Dense(32,activation='relu'), ##HL2
    Dense(1,activation='sigmoid') ##Output Layer
])


In [ ]:
model.summary()

In [ ]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.001)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [ ]:
## compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
## Set up the tensorboard
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [ ]:
## Set up early stopping and TensorBoard callbacks
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
## Train the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, 
                    callbacks=[early_stopping_callback, tensorflow_callback])

In [ ]:
model.save('model.h5')

In [ ]:
## Load TensorBoard Extension
%load_ext tensorboard

In [33]:
%tensorboard --logdir logs/fit/20260219-184927



KeyboardInterrupt

